In [1]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

In [3]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [4]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors, newsgroups_train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [5]:
pred = rf.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary')

0.9209302325581395

In [6]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [21]:
print(c.predict_proba([newsgroups_test.data[0]]))

[[0.314 0.686]]


In [34]:
vectorizer.transform([newsgroups_test.data[idx]]).todense()[0,22729]

0.085218456292116

In [35]:
vectorizer.transform([newsgroups_test.data[idx]]).data

array([0.08521846, 0.54077553, 0.03269163, 0.12612248, 0.05637935,
       0.10603948, 0.08095222, 0.15550487, 0.13434655, 0.07895455,
       0.0983681 , 0.11074493, 0.09806401, 0.08983423, 0.03226976,
       0.20141683, 0.16543838, 0.04053689, 0.24063865, 0.05704586,
       0.0351223 , 0.12557973, 0.05849477, 0.04129161, 0.18433456,
       0.13520646, 0.11169195, 0.15910019, 0.06389102, 0.1296411 ,
       0.08607837, 0.10109536, 0.03408855, 0.05191027, 0.06009027,
       0.06355558, 0.07707962, 0.12493565, 0.030525  , 0.07707962,
       0.03324275, 0.10377733, 0.08696322, 0.18904   , 0.030525  ,
       0.05793467, 0.07695   , 0.15385446, 0.030525  , 0.22257514,
       0.19460537, 0.138911  , 0.21019832, 0.10472637])

In [31]:
vectorizer.transform([newsgroups_test.data[idx]]).indices

array([22729, 22156, 21578, 21396, 20457, 19933, 19720, 19346, 18990,
       18764, 18139, 18001, 17365, 17304, 17234, 17197, 17071, 17064,
       16938, 16306, 15343, 14237, 14215, 13532, 13418, 12550, 12474,
       10958,  9479,  9097,  8895,  8852,  8812,  8789,  7575,  7357,
        7341,  7316,  7106,  6093,  5722,  5535,  5446,  5233,  4850,
        4222,  4103,  4005,  3628,  3547,  2913,  1678,  1567,   101],
      dtype=int32)

In [17]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [9]:
idx = 83
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Document id: 83
Probability(christian) = 0.48
True class: atheism


In [10]:
exp.as_list()

[('Host', -0.12430524944737179),
 ('Posting', -0.11175136265797302),
 ('NNTP', -0.10092660533860623),
 ('edu', -0.0363453180651781),
 ('University', 0.019131853670123496),
 ('an', -0.012656693058354294)]